# Metadata Extraction and Chunking

### [Youtube video covering this notebook](https://youtu.be/JjSCezpZbI0)

### Some important links
- https://unstructured.io/
- https://unstructured-io.github.io/unstructured/index.html
- https://docs.unstructured.io/api-reference/api-services/python-sdk
- https://www.deeplearning.ai/short-courses/preprocessing-unstructured-data-for-llm-applications/


### Semantic Search Vs Hybrid Search

- In RAG, first step is to retrieve documents from a vector database and the basic way to do this is through semantic search which is looking for content that are similar to a query.
- In some cases, 
    - too many matches, 
    - similar content within certain timeframe, 
    - loss of important information ( for example, section information ) can happen. 
    
    To overcome, we can use Hybrid search.  

- Hybrid search allows you to combine similarity search with structured information that you extract from the documents as metadata. We can use metadata as filtering options, most robust retrieval or RAG.

## Setup

In [1]:
%%capture
%pip install "unstructured[all-docs]" unstructured-client watermark

In [2]:
%load_ext watermark

In [3]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [65]:
from IPython.display import display, JSON

import json

from unstructured_client import UnstructuredClient
from unstructured_client.models import shared
from unstructured_client.models.errors import SDKError

from unstructured.partition.pdf import partition_pdf
from unstructured.staging.base import dict_to_elements, elements_to_json

In [7]:
#import unstructured
#import unstructured_client

In [8]:
%watermark --iversions

unstructured_client: 0.22.0
unstructured       : 0.13.7
json               : 2.0.9



## Metadata Extraction

In [9]:
import unstructured.partition

help(unstructured.partition)

Help on package unstructured.partition in unstructured:

NAME
    unstructured.partition

PACKAGE CONTENTS
    api
    auto
    common
    csv
    doc
    docx
    email
    epub
    html
    image
    json
    lang
    md
    model_init
    msg
    odt
    org
    pdf
    pdf_image (package)
    ppt
    pptx
    rst
    rtf
    strategies
    text
    text_type
    tsv
    utils (package)
    xlsx
    xml

FILE
    /Users/sudarshan/Documents/yt-code/youtube-stuffs/data-cleaning/.venv/lib/python3.11/site-packages/unstructured/partition/__init__.py




In [92]:
from unstructured.partition.pdf import partition_pdf

# Specify the path to your PDF file
filename = "data/winter-sports.pdf"

# Call the partition_pdf function
# Returns a List[Element] present in the pages of the parsed pdf document
response = partition_pdf(filename)

In [93]:
response

In [94]:
len(response)

901

In [95]:
element_dict = [el.to_dict() for el in response]
print(json.dumps(element_dict, indent=2))


[
  {
    "type": "ListItem",
    "element_id": "acca646a0bcba83f0edb82ce19dc4949",
    "text": "A Distributed Proofreaders US Ebook *",
    "metadata": {
      "coordinates": {
        "points": [
          [
            91.99219214649183,
            152.85352164572794
          ],
          [
            91.99219214649183,
            164.85352191372795
          ],
          [
            372.83790329871175,
            164.85352191372795
          ],
          [
            372.83790329871175,
            152.85352164572794
          ]
        ],
        "system": "PixelSpace",
        "layout_width": 612,
        "layout_height": 792
      },
      "file_directory": "data",
      "filename": "winter-sports.pdf",
      "languages": [
        "eng"
      ],
      "last_modified": "2024-05-10T09:13:37",
      "page_number": 2,
      "filetype": "application/pdf"
    }
  },
  {
    "type": "NarrativeText",
    "element_id": "6ec7739f853217b1a845380ed110edae",
    "text": "This ebook 

In [108]:
unique_types = set()

for item in element_dict:
    unique_types.add(item['type'])

print(unique_types)

{'NarrativeText', 'ListItem', 'UncategorizedText', 'Title'}


##### Find elements associated with Chapters

In [126]:
[x for x in element_dict if x['type'] == 'Title' and 'winter' in x['text'].lower()]

[{'type': 'Title',
  'element_id': 'bb6a1f21f108211c514bae2404d2f742',
  'text': 'Title: Winter Sports in Switzerland',
  'metadata': {'coordinates': {'points': ((91.99219214649183,
      517.353529786227),
     (91.99219214649183, 529.3535300542271),
     (351.22852985610257, 529.3535300542271),
     (351.22852985610257, 517.353529786227)),
    'system': 'PixelSpace',
    'layout_width': 612,
    'layout_height': 792},
   'file_directory': 'data',
   'filename': 'winter-sports.pdf',
   'languages': ['eng'],
   'last_modified': '2024-05-10T09:13:37',
   'page_number': 2,
   'filetype': 'application/pdf'}},
 {'type': 'Title',
  'element_id': '031880f42be0b69d905158f340cc9bcf',
  'text': 'WINTER SPORTS IN SWITZERLAND',
  'metadata': {'coordinates': {'points': ((179.77735035702676,
      97.98925969642653),
     (179.77735035702676, 163.9892611704264),
     (432.22852749510224, 163.9892611704264),
     (432.22852749510224, 97.98925969642653)),
    'system': 'PixelSpace',
    'layout_width

In [113]:
chapters = [
    "THE SUN-SEEKER",
    "RINKS AND SKATERS",
    "TEES AND CRAMPITS",
    "ICE-HOCKEY",
    "SKI-ING",
    "NOTES ON WINTER RESORTS",
    "FOR PARENTS AND GUARDIANS",
]

In [114]:
chapter_ids = {}
for element in element_dict:
    for chapter in chapters:
        if element["text"] == chapter and element["type"] == "Title":
            chapter_ids[element["element_id"]] = chapter
            break

In [115]:
chapter_ids

{'08ef4b916c44e72dfa21a3b462928b3c': 'THE SUN-SEEKER',
 '83004b57981390b5052bbc077c677e9b': 'RINKS AND SKATERS',
 'd0428d4e418c70676674f93921543ba7': 'TEES AND CRAMPITS',
 'e99335bc0cc4901e83f4ed51b34777a8': 'ICE-HOCKEY',
 '36615698e751eadf28b48d0c96f0384f': 'SKI-ING',
 '06f0ced6df29297c075035fc648700e9': 'NOTES ON WINTER RESORTS',
 '6db9b47269253bef22dfb34790aad59c': 'FOR PARENTS AND GUARDIANS'}

In [129]:
chapter_to_id = {v: k for k, v in chapter_ids.items()}
[x for x in element_dict if x["metadata"].get("parent_id") == chapter_to_id["ICE-HOCKEY"]][0]

{'type': 'NarrativeText',
 'element_id': '295a76c9a7337b0970f95bc27bdee43a',
 'text': 'M\x00\x00\x00 of the Swiss winter-resorts can put into the field a very strong ice-hockey team, and fine teams from other countries often make winter tours there; but the ice-hockey which the ordinary winter visitor will be apt to join in will probably be of the most elementary and unscientific kind indulged in, when the skating day is drawing to a close, by picked-up sides. As will be readily understood, the ice over which a hockey match has been played is perfectly useless for skaters any more that day until it has been swept, scraped, and sprinkled or flooded; and in consequence, at all Swiss resorts, with the exception of St. Moritz, where there is a rink that has been made for the hockey- player, or when an important match is being played, this sport is supplementary to such others as I have spoken of. Nobody, that is, plays hockey and nothing else, since he cannot play hockey at all till the gr

### Load documents into a Vector DB

In [130]:
%pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 4.5 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.7/198.7 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.5/506.5 kB 6.2 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [131]:
import chromadb

In [132]:
client = chromadb.PersistentClient(path="chroma_tmp", settings=chromadb.Settings(allow_reset=True))
client.reset()

INFO: Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


True

In [133]:
collection = client.create_collection(
    name="winter_sports",
    metadata={"hnsw:space": "cosine"}
)

In [134]:
for element in element_dict:
    parent_id = element["metadata"].get("parent_id")
    chapter = chapter_ids.get(parent_id, "")
    collection.add(
        documents=[element["text"]],
        ids=[element["element_id"]],
        metadatas=[{"chapter": chapter}]
    )

/Users/sudarshan/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:14<00:00, 5.94MiB/s]


#### Let's see the elements in VectorDB

In [135]:
results = collection.peek()
print(results["documents"])

['{(a) RfoRK—LboRK {(b) LfoRK—RboRK {(a) RfiRK—LbiRK {(b) LfiRK—RbiRK', '(f) The following figures skated on each foot, namely:—', 'Note.—The head, as already stated, consists of the projection of sixteen stones from one crampit towards the house at the other end of the rink, for each player puts down two stones, and there are eight players. Then when all have played the head is complete, the score is recorded, and the next head is played from the crampit behind the house into which they have just been playing. They “cross over,” that is to say, to the other end of the rink.', 'back two threes—and forward two threes, meet.', 'Here, then, is a rough sketch of the game as it is played, as it', '{(a) RboTfioT—LbiTfoiT {(b) LboTfioT—RbiTfoiT', 'CHAPTER IV', 'II. He must be standing erect with his shoulders and body sideways to the direction of his curve, not facing square down it. In other words, he must, among other things, be travelling not further forward than on the middle of his skate

#### Now, time to do hybrid search with metadata

In [136]:
result = collection.query(
    query_texts=["How many players are on a team?"],
    n_results=2,
    where={"chapter": "ICE-HOCKEY"},
)
print(json.dumps(result, indent=2))

{
  "ids": [
    [
      "081a282afc9095fbc2c76596594c0490",
      "4e83445779c2acfec3d440845954003a"
    ]
  ],
  "distances": [
    [
      0.4859123229980469,
      0.682539701461792
    ]
  ],
  "metadatas": [
    [
      {
        "chapter": "ICE-HOCKEY"
      },
      {
        "chapter": "ICE-HOCKEY"
      }
    ]
  ],
  "embeddings": null,
  "documents": [
    [
      "accuracy of a first-rate team, each member of which knows the play of the other five players. The finer the team, as is always the case, the greater is their interdependence on each other, and the less there is of individual play. Brilliant running and dribbling, indeed, you will see; but as distinguished from a side composed of individuals, however good, who are yet not a team, these brilliant episodes are always part of a plan, and end not in some wild shot but in a pass or a succession of passes, designed to lead to a good opening for scoring. There is, indeed, no game at which team play outwits individual bri

In [145]:
result = collection.query(
    query_texts=["What is Jangfrau?"],
    n_results=2,
    where={"chapter": "ICE-HOCKEY"},
)
print(json.dumps(result, indent=2))

{
  "ids": [
    [
      "999af189fe150b8c751e9dd189c6d901",
      "b70995ddbd9b4f00b95853ac953cbe29"
    ]
  ],
  "distances": [
    [
      0.8623944520950317,
      0.9206119179725647
    ]
  ],
  "metadatas": [
    [
      {
        "chapter": "ICE-HOCKEY"
      },
      {
        "chapter": "ICE-HOCKEY"
      }
    ]
  ],
  "embeddings": null,
  "documents": [
    [
      "It is a wonderful and delightful sight to watch the speed and",
      "But such is not the aspect of the game that will strike the observer who watches the usual pick-up or inter-hotel match on the rink, which generally begins as soon as skaters hear the curfew of the tea-bell. Here will be found the individualist who, sooner than pass when he has once got the puck, would infinitely prefer to fall and be trampled on; and you will see him, while still sitting on the ice, hacking wildly at the beloved india-rubber, in flat contravention of the rule. Common, too, are the \u201cnon-stops\u201d (like Wimbledon trains

In [146]:
result = collection.query(
    query_texts=["What is Jangfrau?"],
    n_results=2,
    where={"chapter": "THE SUN-SEEKER"},
)
print(json.dumps(result, indent=2))

{
  "ids": [
    [
      "a2313f32eead49a43c4d1e41066bdeac",
      "6c645fef54fc43754627dbe221205b96"
    ]
  ],
  "distances": [
    [
      0.5990142822265625,
      0.7780079245567322
    ]
  ],
  "metadatas": [
    [
      {
        "chapter": "THE SUN-SEEKER"
      },
      {
        "chapter": "THE SUN-SEEKER"
      }
    ]
  ],
  "embeddings": null,
  "documents": [
    [
      "The Jungfrau is and will always be my mistress among mountains,",
      "as a first charge, and after they have been stowed, the mere necessities of life, like clothes and dressing-case, as opposed to its joys, fill the rest. Even in the harbour the steamer sways with the back-wash of the heavy seas outside, and the mooring-ropes squeak and strain to its unease. I stick in the narrow gang-plank that conducts in precipitous incline to the deck (at least the corner of my suit-case does, which is part of my identity); a faint and awful smell of red plush sofas and cold beef comes up from the stairs leading 

In [147]:
# without providing the specific chapter to go for
result = collection.query(
    query_texts=["What is Jangfrau?"],
    n_results=2,
    #where={"chapter": "THE SUN-SEEKER"},
)
print(json.dumps(result, indent=2))

{
  "ids": [
    [
      "a2313f32eead49a43c4d1e41066bdeac",
      "f22253bd2860aca62523bd31739c3207"
    ]
  ],
  "distances": [
    [
      0.5990142822265625,
      0.745063304901123
    ]
  ],
  "metadatas": [
    [
      {
        "chapter": "THE SUN-SEEKER"
      },
      {
        "chapter": ""
      }
    ]
  ],
  "embeddings": null,
  "documents": [
    [
      "The Jungfrau is and will always be my mistress among mountains,",
      "CHAPTER VII"
    ]
  ],
  "uris": null,
  "data": null
}


## Now, let's do Chunking by Title 
- Instead of the de-facto chunking we can dictate how to do the chunking of documents.

In [139]:
from unstructured.chunking.title import chunk_by_title

In [140]:
chunks = chunk_by_title(
    response,
    combine_text_under_n_chars=100,
    max_characters=3000,
)

In [141]:
JSON(json.dumps(chunks[0].to_dict(), indent=2))

<IPython.core.display.JSON object>

In [143]:
len(element_dict)

901

In [144]:
len(chunks)

199

### Now, you can use Unstructured API too

#### [Unstructured API link](https://unstructured-io.github.io/unstructured/api.html)
- For SAAS API, visit this [link](https://unstructured.io/api-key-hosted)
- For Free Unstructured API, visit this [link](https://docs.unstructured.io/api-reference/api-services/free-api)

In [55]:
%pip install python-dotenv


[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [56]:
import os
from dotenv import load_dotenv, find_dotenv

# Load the .env file
load_dotenv(find_dotenv())

True

In [105]:
free_api_key_auth = os.environ.get('free_api_key_auth')
client = UnstructuredClient(
    api_key_auth=free_api_key_auth
    #api_key_auth=saas_api_key_auth,
    #server_url=saas_server_url,
)

In [106]:
with open(filename, "rb") as f:
    files=shared.Files(
        content=f.read(),
        file_name=filename,
    )

req = shared.PartitionParameters(files=files)

In [107]:
try:
    resp = client.general.partition(req)
except SDKError as e:
    print(e)

API error occurred: Status 402
{"detail":"The organization associated with this API key has reached Freemium API monthly quota cap. Please sign up for the Paid API here https://unstructured.io/api-key-hosted."}


###### You can see that there is limit for the Free API, you might consider using SAAS API if needed.